

# ML Study Jam Exercise 3

## Setup Kaggle Library

In [ ]:
# install kaggle libary
%pip install -q kaggle

In [ ]:
# upload Kaggle API Credentials
from google.colab import files

files.upload()

In [ ]:
# add kaggle API credentials to root
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# create a new folder to save dataset
! mkdir ./kaggleDataset

# set Directory as current directory
%cd ./kaggleDataset

In [ ]:
# list kaggle datasets
! kaggle datasets list

## Computer Vision

### Understanding CNN

In [ ]:
# import libraries

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras import optimizers

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
from  shutil import copyfile
import random

In [ ]:
# create new dir for dataset
! mkdir ./dogs-vs-cats

# set Directory as current directory
%cd ./dogs-vs-cats

In [ ]:
# download the dataset
!kaggle datasets download -d biaiscience/dogs-vs-cats

In [ ]:
# Complete path to storage location of the .zip file of data
zip_path = '/content/kaggleDataset/dogs-vs-cats/dogs-vs-cats.zip'
# Check current directory (be sure you're in the directory where Colab operates: '/content')
os.getcwd()
# Copy the .zip file into the present directory
!cp '{zip_path}' .
# Unzip quietly
!unzip -q 'dogs-vs-cats.zip'
# View the unzipped contents in the virtual machine
os.listdir()

In [ ]:
# Set up file paths
TRAIN_PATH='/content/kaggleDataset/dogs-vs-cats/train/train'
TEST_PATH='/content/kaggleDataset/dogs-vs-cats/test/test'

In [ ]:
# Function to extract class label from file name
def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat': return 'cat'
    elif word_label == 'dog': return 'dog'

# Prepare dataframes
train_images = os.listdir(TRAIN_PATH)

train_labels = [label_img(img) for img in train_images]

train_df = pd.DataFrame({
    'filename': train_images,
    'category': train_labels
})

# Shuffle the dataframe and split into train and validation
train_df = train_df.sample(frac=1).reset_index(drop=True)
validation_df = train_df.iloc[:250]  # first 250 for validation
train_df = train_df.iloc[250:].reset_index(drop=True)  # rest for training

In [ ]:
train_df = train_df.iloc[:1000].reset_index(drop=True) # keep first 1000 images only
train_df

In [ ]:
train_image_generator = ImageDataGenerator(rescale=1./255,
                                           rotation_range=45,
                                           width_shift_range=.15,
                                           height_shift_range=.15,
                                           horizontal_flip=True,
                                           zoom_range=0.3
                                           ) # Generator for our training data

In [ ]:
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [ ]:
train_data_gen = train_image_generator.flow_from_dataframe(train_df,
                                                           directory=TRAIN_PATH,
                                                           x_col='filename',
                                                           y_col='category',
                                                           shuffle=True,
                                                           target_size=(150, 150),
                                                           class_mode='binary',
                                                           batch_size=256
                                                           )
val_data_gen = validation_image_generator.flow_from_dataframe(validation_df,
                                                              directory=TRAIN_PATH,
                                                              x_col='filename',
                                                              y_col='category',
                                                              shuffle=False,  # Keep data in same order as labels
                                                              class_mode='binary',
                                                              target_size=(150, 150),
                                                              batch_size=256
                                                              )

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

In [ ]:
sample_training_images, sample_training_labels = next(train_data_gen)
plotImages(sample_training_images[:5])

In [ ]:
import datetime

# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
#Creating a simple CNN model

model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(150, 150 ,3)),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

In [ ]:
# print model details
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(train_data_gen, epochs=2, validation_data=val_data_gen, callbacks=[tensorboard_callback])

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
model.save('cats_and_dogs_model.h5')

In [ ]:
def plotFilters(conv_filter):
    fig, axes = plt.subplots(1, 3, figsize=(5,5))
    axes = axes.flatten()
    for img, ax in zip( conv_filter, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
#Visualizing the filters
for layer in model.layers:
    if 'conv' in layer.name:
        weights, bias= layer.get_weights()
        print(layer.name, weights.shape)
         #normalize filter values between  0 and 1 for visualization
        f_min, f_max = weights.min(), weights.max()
        filters = (weights - f_min) / (f_max - f_min)
        print(weights.shape[3])
        filter_cnt=1
        #plotting all the filters
        for i in range(filters.shape[3]):
            #get the filters
            filt=filters[:,:,:, i]
            #plotting ecah channel
            for j in range(filters.shape[0]):
                ax= plt.subplot(filters.shape[3], filters.shape[0], filter_cnt  )
                ax.set_xticks([])
                ax.set_yticks([])
                plt.imshow(filt[:,:, j])
                filter_cnt+=1
        plt.show()


In [ ]:
img_path='/content/kaggleDataset/dogs-vs-cats/train/train/cat.1341.jpg'
# Let's define a new Model that will take an image as input, and will output
# intermediate representations for all layers in the previous model after
# the first.
successive_outputs = [layer.output for layer in model.layers[1:]]

#visualization_model = Model(img_input, successive_outputs)
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

img = load_img(img_path, target_size=(150, 150))  # this is a PIL image

x   = img_to_array(img)                           # Numpy array with shape (150, 150, 3)
x   = x.reshape((1,) + x.shape)                   # Numpy array with shape (1, 150, 150, 3)

# Rescale by 1/255
x /= 255.0

# Let's run our image through our network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers]

# -----------------------------------------------------------------------
# Now let's display our representations
# -----------------------------------------------------------------------
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  print(feature_map.shape)
  if len(feature_map.shape) == 4:

    #-------------------------------------------
    # Just do this for the conv / maxpool layers, not the fully-connected layers
    #-------------------------------------------
    n_features = feature_map.shape[-1]  # number of features in the feature map
    size       = feature_map.shape[ 1]  # feature map shape (1, size, size, n_features)

    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))

    #-------------------------------------------------
    # Postprocess the feature to be visually palatable
    #-------------------------------------------------
    for i in range(n_features):
      x  = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std ()
      x *=  64
      x += 128
      x  = np.clip(x, 0, 255).astype('uint8')
      display_grid[:, i * size : (i + 1) * size] = x # Tile each filter into a horizontal grid

    #-----------------
    # Display the grid
    #-----------------

    scale = 20. / n_features
    plt.figure( figsize=(scale * n_features, scale) )
    plt.title ( layer_name )
    plt.grid  ( False )
    plt.imshow( display_grid, aspect='auto', cmap='viridis' )

### Transfer Learning

In [ ]:
import numpy as np
import pandas as pd # Data Frame processing
import tensorflow as tf # Backend library for neural nets processing
from keras import models, layers # Framework for neural nets creating
import os, shutil # filesystem operations
from os import listdir
from os.path import isfile, join
import random
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import matplotlib.pyplot as plt
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix
from keras.applications import Xception
from keras.applications import VGG19
from keras.applications import ResNet50
from keras.applications import MobileNet
from keras.callbacks import EarlyStopping
from keras.layers import Dense, GlobalAveragePooling2D


%matplotlib inline

In [ ]:
# GPU checking
device_names = tf.test.gpu_device_name()
device_names

In [ ]:
# Set up file paths
TRAIN_PATH='/content/kaggleDataset/dogs-vs-cats/train/train'
TEST_PATH='/content/kaggleDataset/dogs-vs-cats/test/test'

In [ ]:
target_size = (150,150)
batch_size = 256

#Fetching train data and validation data and processing the data
train_datagen = ImageDataGenerator(rescale = 1.00 / 255.0)
val_datagen = ImageDataGenerator(rescale = 1.00 / 255.0)
test_datagen = ImageDataGenerator(rescale = 1.00 / 255.0)

train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=TRAIN_PATH,
                                                    x_col='filename',
                                                    y_col='category',
                                                    shuffle=True,
                                                    target_size=target_size,
                                                    class_mode='binary',
                                                    batch_size=batch_size
                                                    )

validation_generator = val_datagen.flow_from_dataframe(validation_df,
                                                      directory=TRAIN_PATH,
                                                      x_col='filename',
                                                      y_col='category',
                                                      shuffle=False,  # Keep data in same order as labels
                                                      class_mode='binary',
                                                      target_size=target_size,
                                                      batch_size=batch_size
                                                      )

In [ ]:
def _round(vec, threshold):
    output = []
    for i in vec:
        if i >= threshold:
            output.append(np.ceil(i))
        else:
            output.append(np.floor(i))
    return np.array(output)

def plot_confusion_matrix(cm,
                      classes,
                      normalized=False,
                      title=None,
                      cmap=plt.cm.Blues,
                      size=(2,2)):
    fig, ax = plt.subplots(figsize=size)
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)

In [ ]:
input_shape = (150, 150, 3)
model_vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
model_vgg19.trainable = False
model_vgg19.summary()

In [ ]:
# add new classifier layers
from keras.models import Model
flat1 = model_vgg19.layers[-1].output
flat1 = GlobalAveragePooling2D()(flat1)
class1 = layers.Dense(64, activation='relu')(flat1)
output = layers.Dense(1, activation='sigmoid')(class1)

# define new model
model = Model(inputs=model_vgg19.inputs, outputs=output)
# summarize
model.summary()

In [ ]:
model.compile(
  loss = "binary_crossentropy",
  optimizer='adam',
  metrics = ["acc"]
)

In [ ]:
#Training the model with train data and judging this training with validation data
history = model.fit(
    train_generator,
    batch_size=batch_size,
    epochs = 5,
    validation_data = validation_generator)

In [ ]:
#Train accuracy and validation accuracy vs epoch graph

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, label='Training acc')
plt.plot(epochs, val_acc, label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
# Get predictions

y_pred = model.predict(validation_generator)

y_pred = _round(y_pred, 0.5)
y_true = validation_generator.classes
mcm = multilabel_confusion_matrix(y_true, y_pred)
cmn = confusion_matrix(y_true, y_pred, normalize='true')
print(cmn)

In [ ]:
labels = ['dog', 'cat']
plot_confusion_matrix(cmn,
                      labels,
                      normalized=True,
                      title="Model Performance",
                      cmap=plt.cm.Blues,
                      size=(2,2))

## Natural Processing Language

### Seq2Seq Model
English to French Translator



In [ ]:
from keras.models import Model
from keras.layers import Input, CuDNNLSTM, Dense
import numpy as np
import pandas as pd
import os

from numpy.random import seed
seed(1)

In [ ]:
# create new dir for dataset
! mkdir /content/kaggleDataset/frenchenglish-bilingual-pairs

# set Directory as current directory
%cd /content/kaggleDataset/frenchenglish-bilingual-pairs

In [ ]:
# download the dataset
!kaggle datasets download -d jannesklaas/frenchenglish-bilingual-pairs

In [ ]:
# Complete path to storage location of the .zip file of data
zip_path = '/content/kaggleDataset/frenchenglish-bilingual-pairs/frenchenglish-bilingual-pairs.zip'
# Check current directory (be sure you're in the directory where Colab operates: '/content')
os.getcwd()
# Copy the .zip file into the present directory
!cp '{zip_path}' .
# Unzip quietly
!unzip -q 'frenchenglish-bilingual-pairs.zip'
# View the unzipped contents in the virtual machine
os.listdir()

In [ ]:
batch_size = 256  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

# Path to the data txt file on disk.
data_path = '/content/kaggleDataset/frenchenglish-bilingual-pairs/fra-eng/fra.txt'

In [ ]:
# read in the datafile
df = pd.read_csv(data_path,delimiter='\t', header=None)
df = df.rename(columns={0: 'English', 1: 'French'})
df

In [ ]:
df.head()

#### Vectorizing the data

In [ ]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

# Loop over lines
lines = open(data_path).read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    # Input and target are split by tabs
    # English TAB French
    input_text, target_text = line.split('\t')

    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)

    # Create a set of all unique characters in the input
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)

    # Create a set of all unique output characters
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [ ]:
# Make sure we achieve the same order in our input chars (sorted based on ASCII)
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters) # aka size of the english alphabet + numbers, signs, etc.
num_decoder_tokens = len(target_characters) # aka size of the french alphabet + numbers, signs, etc.
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)

In [ ]:
# Create character tokenizer
# The index maps a character to a number
input_token_index = {char: i for i, char in enumerate(input_characters)}
target_token_index = {char: i for i, char in enumerate(target_characters)}

In [ ]:
input_token_index

In [ ]:
# Demo character tokenization
for c in 'the cat sits on the mat':
    print(input_token_index[c], end = ' ')
    print(c)

In [ ]:
# Get longest sequences length
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

In [ ]:
# encoder_input_data is a 3D array of shape (num_pairs, max_english_sentence_length, num_english_characters)
# containing a one-hot vectorization of the English sentences.

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens),dtype='float32')

# decoder_input_data is a 3D array of shape (num_pairs, max_french_sentence_length, num_french_characters)
# containg a one-hot vectorization of the French sentences.

decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')

# decoder_target_data is the same as decoder_input_data but offset by one timestep.
# decoder_target_data[:, t, :] will be the same as decoder_input_data[:, t + 1, :]

decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')

In [ ]:
# Loop over input texts
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):

    # Loop over each char in an input text
    for t, char in enumerate(input_text):
        # Create one hot encoding by setting the index to 1
        encoder_input_data[i, t, input_token_index[char]] = 1.

    # Loop over each char in the output text
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.

        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [ ]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens), name = 'encoder_inputs')

# The return_state contructor argument, configuring a RNN layer to return a list
# where the first entry is the outputs and the next entries are the internal RNN states.
# This is used to recover the states of the encoder.
encoder = CuDNNLSTM(latent_dim, return_state=True, name = 'encoder')

encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens), name = 'decoder_inputs')

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = CuDNNLSTM(latent_dim, return_sequences=True, return_state=True, name = 'decoder_lstm')

# The inital_state call argument, specifying the initial state(s) of a RNN.
# This is used to pass the encoder states to the decoder as initial states.
# Basically making the first memory of the decoder the encoded semantics
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax', name = 'decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.summary()

In [ ]:
# Run training
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')
history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,batch_size=batch_size,epochs=epochs,validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,7))
a, = plt.plot(history.history['loss'],label='Training Loss')
b, = plt.plot(history.history['val_loss'],label='Validation Loss')
plt.legend(handles=[a,b])
plt.show()

In [ ]:
# Define encoder model
encoder_model = Model(encoder_inputs, encoder_states)

In [ ]:
# Define decoder model

# Inputs from the encoder
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))

# Create a combined memory to input into the decoder
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Decoder
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

# Predict next char
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
# The model takes in the encoder memory plus it's own memory as an input and spits out
# a prediction plus its own memory to be used for the next char
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [ ]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = {i: char for char, i in input_token_index.items()}
reverse_target_char_index = {i: char for char, i in target_token_index.items()}

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''

    # Loop untill we recieve a stop sign
    while not stop_condition:
        # Get output and internal states of the decoder
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Get the predicted token (the token with the highest score)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        # Get the character belonging to the token
        sampled_char = reverse_target_char_index[sampled_token_index]
        # Append char to output
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
my_text = 'Thanks'
placeholder = np.zeros((1,len(my_text)+10,num_encoder_tokens))

In [ ]:
for i, char in enumerate(my_text):
    print(i,char, input_token_index[char])
    placeholder[0,i,input_token_index[char]] = 1

In [ ]:
decode_sequence(placeholder)

## Deep Learning

In [ ]:
# import libraries
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
# explore the dataset
print("Number of training examples:", x_train.shape[0])
print("Number of test examples:", x_test.shape[0])

In [ ]:
# visualize the first training example
import matplotlib.pyplot as plt

for i in range(0,5):
  plt.imshow(x_train[i], cmap='gray')
  plt.title('Ground Truth : {}'.format(y_train[i]))
  plt.show()


In [ ]:
# Reshape and normalize the input data
x_train = x_train.reshape(-1, 784) / 255.0
x_test = x_test.reshape(-1, 784) / 255.0

In [ ]:
# Convert the labels to one-hot encoded vectors
y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)

In [ ]:
# Create the deep learning model
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(784,)))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [ ]:
# print the model summary
model.summary()

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

In [ ]:
# use history object to plot the training and validation accuracy for each epoch
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# use history object to plot the training and validation loss for each epoch
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()